# Généralisation

In [1]:
import numpy as np
from tqdm import tqdm 
from sklearn.datasets import make_circles
from sklearn.metrics import log_loss, accuracy_score
import matplotlib.pyplot as plt

![w&b](multi_layer_png2/w&b.png "w&b")

In [2]:
def initialisation(dimensions):
    parametres = {} # On initialise un dictionnaire vide
    C = len(dimensions) # On récupère la len du tableau dimensions, car nous voulons accèder aux index

    for c in range(1, C): # On commence à 1 la boucle car la première entrée sont les entrés X et on arréte à C pour ne pas prendre le neuronne final (car python arrete à -1).
        parametres['W' + str(c)] = np.random.randn(dimensions[c], dimensions[c - 1]) # Nouvelle clée, accès aux valeur de dimensions, création de la matrice.
        parametres['b' + str(c)] = np.random.randn(dimensions[c], 1)
    return parametres

In [3]:
parametres = initialisation([2, 32, 32, 1]) # [entrés X, couche 1, couche 2, …, couche n]

for key, val in parametres.items():
    print(key, val.shape)


W1 (32, 2)
b1 (32, 1)
W2 (32, 32)
b2 (32, 1)
W3 (1, 32)
b3 (1, 1)


![z&c](multi_layer_png2/z&c.png "z&c")

In [4]:
def forward_propagation(X, parametres):
    C = len(parametres) // 2 # On divise avec 2 '/' pour retourner un nombre entier. Divise par deux pour obtenir le nombre de couche car il y a W et b dans la longueure.
    activations = {'A0' : X} # Car la première couche ce calcul avec les X, pas avec n-1.

    for c in range(1, C + 1): # Python s’arrete 1 nombre avant la fin de la boucle, donc C + 1 pour s’arréter à C
        Z = parametres['W' + str(c)] @ activations['A' + str(c - 1)] + parametres['b' + str(c)]
        activations['A' + str(c)] = 1 / (1 + np.exp(-Z)) # On append directement dans la boucel les A

    return activations

In [5]:
activations = forward_propagation(X, parametres)
print(X.shape)
for key, val in activations.items():
    print(key, val.shape)

NameError: name 'X' is not defined

In [ ]:
def back_propagation(X, y, parametres, activations):
    A1 = activations['A1']
    A2 = activations['A2']
    W2 = parametres['W2']

    m = y.shape[1]

    dZ2 = A2 - y
    dW2 = 1 / m * dZ2 @ A1.T
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True) # On met le parametre keepdims true pour garder les dimensions, car avec le broadcasting que nous executerons, cela peut casser les dims de la matrice.   

    dZ1 = np.dot(W2.T, dZ2) * A1 * (1 - A1)
    dW1 = 1 / m * dZ1 @ X.T
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)

    gradients = {
        'dW1' : dW1,
        'db1' : db1,
        'dW2' : dW2,
        'db2' : db2
    }

    return gradients

![back_propagation](multi_layer_png2/back_propagation.png "back_propagation")

In [ ]:
def back_propagation(y, activations, parametres):
    m = y.shape[1]
    C = len(parametres) // 2
    
    dZ = activations['A' + str(C)] - y
    gradients = {}

    for c in reversed(range(1, C + 1)): 
        gradients['dW' + str(c)] = 1 / m * dZ @ activations['A' + str(c - 1)].T
        gradients['db' + str(c)] = 1 / m * np.sum(dZ, axis=1, keepdims=True)
        if c > 1:
            dZ = np.dot(parametres['W' + str(c)].T, dZ) * activations['A' + str(c - 1)] * (1 - activations['A' + str(c - 1)])

    return gradients

In [ ]:
gradients = back_propagation(y, activations, parametres)

for key, val in gradients.items():
    print(key, val.shape)

dW3 (1, 32)
db3 (1, 1)
dW2 (32, 32)
db2 (32, 1)
dW1 (32, 2)
db1 (32, 1)


![update](multi_layer_png2/update.png "update")

In [ ]:
def update(parametres, gradients ,learning_rate): 
    C = len(parametres) // 2

    for c in range(1, C + 1):
        parametres['W' + str(c)] = parametres['W' + str(c)] - learning_rate * gradients['dW' + str(c)]
        parametres['b' + str(c)] = parametres['b' + str(c)] - learning_rate * gradients['db' + str(c)]

    return parametres